# <center> Résultats de la macro de quantification de chaque type cellulaire </center>

**Lancer la macro de quantification. Si vous avez besoin des étapes pour lancer la macro lire la section Lancer la macro de comptage ou la macro de classification de la fiche utilisateur**. Vous venez d'utiliser la macro de comptage sur imageJ et vous obtenez par puits deux fichiers au format CSV contenant les résultats. Les résultats regroupent le comptage pour chaque image du nombre d'astrocytes, nombre d'oligodendrocytes et nombre de noyaux, le second fichier CSV contient les airs des cytoplasmes d'oligodendrocytes et d'astrocytes. Ces informations sont traitées dans ce script pour obtenir le pourcentage d'astrocytes et d'oligodendrocytes par images mais également pour l'ensemble du puits. Ces pourcentages sont visualisés à l'aide de graphiques notamment de bar plot. Cette partie du script nécessite la bibliothèque pandas qui permet de manipuler et d'analyser des données. Le fichier ouvert est donc un data frame nommé *data* dans ce script. Affichage des dimensions ainsi que l'ensemble des informations du data frame.


## Importer les bibliothèques et modification des variables
**Veuillez exécuter la cellule suivante qui permet d'importer les bibliothèques nécessaires pour l'utilisation de ce script et la modification des chemins.** Les variables suivantes représentent les chemins contenant les fichiers CSV produits par les macros et qui sont analysés par ce script. Certaine variables représentent le chemin du répertoire qui contiendra les fichier csv produit par ce script. Pour savoir quel chemin mettre pour chaque variable lire les commentaires qui sont associés à la variable.

## Modifier les variables suivantes (en bleu) 
### Variables pour la macro de quantification
- <font color='blue'> **rep_result_macro** </font> par le chemin qui contient les fichiers csv de la macro de quantification des différents types cellulaires.
- <font color='blue'> **nom_csv_r** </font> le nom choisi pour nommé les fichiers csv contenant le **comptage** obtenu avec la macro de quantification des différents types cellulaires.
- <font color='blue'> **nom_csv_a** </font> le nom choisi pour nommé les fichiers csv contenant le **air** obtenu avec la macro de quantification des différents types cellulaires.
- <font color='blue'> **rep_result_comptage** </font> le chemin du répertoire qui contiendra le fichier csv produit par le script.
- <font color='blue'> **name_result_comptage** </font> le nom du fichier csv produit par le script pour la comptage des O4 et GFAP.

### Variables pour comparer la quantification chez les patients et contrôles pour chaque traitement 
- <font color='blue'>**Controle, Malade, Controle1** et **Malade1**</font> La liste *Controle* contient le pourcentage d'astrocytes pour les contrôles dans les conditions avec GMCSF, avec IL4 et avec LPS. La liste *Malade* contient le pourcentage d'astrocytes chez les malades pour les conditions avec GMCSF, avec IL4 et avec LPS. Le même processus est réalisé pour le pourcentage d'oligodendrocytes.

In [1]:
import os
import glob
import pandas as pd
import pprint
import matplotlib.pyplot as pyplot

# Macro de quantification des différents types cellulaire
# Chemin contenant les fichiers csv obtenus avec la macro de classification
rep_result_macro =r"C:\Users\naima.ammiche\Desktop\Données - traitement images\new result comptage"
# Nom choisi pour les fichiers comptage lors du lancement de la macro
nom_csv_r ="resultat"
# Nom choisi pour les fichiers air lors du lancement de la macro
nom_csv_a ="air"
# Enregistrement du fichiers csv produit
# Répertoire ou sera présant le fichier csv crée à la fin du script
rep_result_comptage =r"C:\Users\naima.ammiche\Desktop\Données - traitement images"
# Nom du fichier csv qui sera produit à la fin du scirpt
name_result_comptage = "resultat_analyse_des_puits"


# Pourcentage d'astrocytes et d'oligodendrocytes pour les contrôles et malades en fonction du surnageant
# % Astrocytes
Controle = [18.2987, 11.36524, 42.8085] # [% avec GMCSF, % avec IL4, % avec LPS]
Malade = [11.07185, 9.902478, 35.43933]  # [% avec GMCSF, % avec IL4, % avec LPS]
# % Oligodendrocytes
Controle1 = [29.54334, 38.62412, 24.42729]  # [% avec GMCSF, % avec IL4, % avec LPS]
Malade1 = [27.644, 25.87165, 23.0211]  # [% avec GMCSF, % avec IL4, % avec LPS]

## Analyse des résultats
Avant de commencer l'analyse qui permet la quantification de chaque type cellulaire (astrocytes, oligodendrocytes et noyaux), l'air des cytoplasmes d'astrocytes et d'oligodendrocytes ainsi que leurs pourcentages et nombre de champs pour chaque puits. Il est nécessaire d'initialiser ces variables avec une liste vide **GFAP, O4, h, pourcentage_O4, pourcentage_GFAP, aire_O4, aire_GFAP, nb_champs et index**, ces listes sont remplie au fur et à mesure de l'analyse.

Ouvrir le répertoire contenant l'ensemble des fichiers csv produits par la macro de comptage. Récupérer les fichiers csv dont le nom commence par le **nom_csv_r** choisit *Exemple 'Resultat* dans la liste **csv_files**.  Récupérer les fichiers csv dont le nom commence par le **nom_csv_a** choisit *Exemple 'Air'* dans la liste **csv_files_air**. Parcourir la liste des fichiers csv et pour chaque fichier csv récupérer le nom du puits et l'ajouter à la liste **index** s'il n'est pas présent. Parcourir ensuite la liste **index** contenant l'ensemble des puits analyser donc *A01* puis *A04* etc. et parcourir en même temps les fichiers csv.

Ouvrir le fichier csv *Exemple ResultatA01* en data frame et afficher ces dimensions. Récupérer depuis se data frame le nombre de noyaux, d'oligodendrocytes et d'astrocytes et calculer le pourcentage des oligodendrocytes et astrocytes. Récupérer également le nombre de champs c'est-à-dire le nombre de lignes divisé par trois. Ces valeurs sont ajoutées aux listes correspondantes **GFAP, O4, h, pourcentage_O4, pourcentage_GFAP et nb_champs**. 

Ouvrir le fichier csv *Exemple AirA01* en data frame et afficher ces dimensions. Récupérer depuis ce data frame l'air des cytoplasmes d'oligodendrocytes et d'astrocytes et l'ajouter aux listes corrspondantes **aire_O4 et aire_GFAP**. Pour chaque puits afficher le nombre d'astrocytes, d'oligodendrocytes, de nombre leurs pourcentages.

In [ ]:
# Initialisation
GFAP = []
O4 = []
h = []
pourcentage_O4 = []
pourcentage_GFAP = []
aire_O4 = []
aire_GFAP = []
nb_champs = []

# Légende: les puits analysé
index = []

In [ ]:
# Répertoire contenant les puits à analyser
print(f"Répertoire contenant les puits à analyser :\n{rep_result_macro}\n")

# Comptage : récuper que les fichiers commençant par le nom choisi
csv_regex = os.path.join(rep_result_macro, nom_csv_r+"*.csv") # Une chaîne de caractère contenant nom des fichiers à sélectionner
csv_files = glob.glob(csv_regex) # Liste de l'ensemble des fichiers CSV

# Air : récuper que les fichiers commençant par le nom choisi
csv_regex_air = os.path.join(rep_result_macro, nom_csv_a+"*.csv") # Une chaîne de caractère contenant nom des fichiers à sélectionner
csv_files_air  = glob.glob(csv_regex_air) # Liste de l'ensemble des fichiers CSV

# Pour chaque répertoire (= pour chaque puits)
for i in range(len(csv_files)):
    
    # Comptage : récupérer le nom du puits (Ex: A01) et le chemin pour ouvrir le fichier associé à ce puits 
    chemin_split = csv_files[i].split("\\")
    nom_csv = chemin_split[-1]
    nom_puits = nom_csv[-7:-4]
    puits = rep_result_macro + '\\' +nom_csv
    
    # Air : récupérer le nom du puits (Ex: A01) et le chemin pour ouvrir le fichier associé à ce puits 
    chemin_split_air = csv_files_air[i].split("\\")
    nom_csv_air = chemin_split_air[-1]
    nom_puits_air = nom_csv_air[-7:-4]
    puits_air = rep_result_macro + '\\' +nom_csv_air
    
    # Si le puits n'est pas encore analyser l'ajouter à la liste index et réaliser l'analyse
    if nom_puits not in index:
        print(f"\nPuits à analyser : {nom_puits}")         
        
        # Ouvrir le data frame comptage
        try: 
            # Ouvrir le fichier csv associé au puits
            data = pd.read_csv(puits)
            
            
            # Ajouter le nombre de champs
            nb_champs.append(data.shape[0]/3)

            # Afficher les dimensions du data frame contenant le comptage des astrocytes, oligodendrocytes et noyaux pour chaque puits
            print(f"Les dimensions du data frame de comptage : {data.shape} : {data.shape[0]} lignes et {data.shape[1]} colonnes")
           
            
            # Ajout du nombre astrocytes, oligodendrocytes et noyaux aux liste GFAP, O4 et h
            GFAP_puits = data.groupby(['Slice']).sum()["Count"]["GFAPcell"] # Nombre d'astrocytes dans le puits
            O4_puits = data.groupby(['Slice']).sum()["Count"]["O4cell"] # Nombre d'oligodendrocytes dans le puits
            GFAP.append(GFAP_puits)
            O4.append(O4_puits)
            h.append(total)
            
            
            # Ajouter le % astrocytes et % oligodendrocytes pour chaque puits
            # Nombre total de cellules dans le puits (= nombre de noyaux)
            total = data.groupby(['Slice']).sum()['Count'].loc['h+']
            
            # % d'astrocytes
            astrocytes = GFAP_puits/total*100 
            pourcentage_GFAP.append(astrocytes)
            
            # % d'oligodendrocytes
            oligodendrocytes = O4_puits/total*100
            pourcentage_O4.append(oligodendrocytes)
            
            
            
            
        except :
            s='comptage'
            #print(f"Le chemin pour ouvrir le fichier est incorrect {puits}")
            
            
        # Ouvrir le data frame air
        try :
            # Ouvrir le fichier csv
            data_air = pd.read_csv(puits_air)
            
            # Ajout air des oligodendrocytes
            aire_O4.append(data_air.groupby(by="Label").sum()["Area"]["O4mesure"])
            
            # Ajout air des astrocytes
            aire_GFAP.append(data_air.groupby(by="Label").sum()["Area"]["GFAPmesure"])
            
            # Afficher un message pour l'utilisateur
            print(f"Les dimensions du data frame air des cytoplasmes : {data_air.shape} : {data_air.shape[0]} lignes et {data_air.shape[1]} colonnes\n% GFAP {astrocytes:.2f}\n% O4 {oligodendrocytes:.2f}\nO4 {O4_puits}\nGFAP {GFAP_puits}\nH+ {total}")

        except:
            s='air'
            #print("Le chemin pour ouvrir le fichier est incorrect")
        
        # Nom du puits à analyser ajouter à la liste index
        index.append(csv_files[i][-7:-4])
        
    # Si les puits sont déjà analysé prévenir l'utilisateur avec le message suivant    
    else :
        i=1
        
    
print(f"\nLes puits dont l'analyse est terminé:{index}\nAire O4: {aire_O4}\nAire GFAP: {aire_GFAP}\nGFAP: {GFAP}\nO4: {O4}\nH+: {h}\n%GFAP:{pourcentage_GFAP}\n%O4: {pourcentage_O4}\nNombre de champs : {nb_champs}")

## Production du data frame du comptage des O4 et GFAP
Ce data frame regroupe les informations sur l'air des cytoplames d'oligodendrocytes, air des cytoplasmes d'astrocytes, le nombre d'astrocytes, d'oligodendrocytes et de noyaux, le pourcentage d'astrocytes, le pourcentage d'oligodendrocytes et le nombre de champs pour chaque puits. En lignes sont représentés les puits et en colonnes ces caractéristiques.

In [ ]:
# Production du dataframe
data_result_comptage = pd.DataFrame({'aire O4': aire_O4,
                   'aire GFAP': aire_GFAP,
                   'GFAP': GFAP,
                   'O4':O4,
                   'H+':h,
                   '% GFAP':pourcentage_GFAP,
                   '% O4':pourcentage_O4,
                   'nb champs':nb_champs}, index=index)

# Sauvegarde du data frame au format csv
data_result_comptage.to_csv(os.path.join(rep_result_comptage, name_result_comptage+".csv"))

# Visualisation du data frame
data_result_comptage

###  Visualisation graphique du pourcentage d'astrocytes et d'oligodendrocytes pour les contrôles et malades en fonction du surnageant
La fonction DataFrame permet de créer un data frame avec les listes "Controle" et "Malade", la fonction plot.bar permet de représenter le data frame sous forme de bar plot. Représentation graphique du pourcentage d'astrocytes (premier graphique) et d'oligodendrocytes (second graphique) pour les contrôles (en bleu) et malades (en orange) en fonction du surnageant GMCSF, IL4 et LPS.

In [ ]:
# légende pour les trois traitements
index = ['GMCSF', 'IL4', 'LPS'] 

# % Astrcoytes
df = pd.DataFrame({'Contrôle': Controle,
                   'Malade': Malade}, index=index)
ax = df.plot.bar(rot=0)

# % Oligodendrocytes
df1 = pd.DataFrame({'Contrôle': Controle1,
                   'Malade': Malade1}, index=index)
ax = df1.plot.bar(rot=0)